/home/lwecke/Github/BlackboxOptimisation/finalEnv/bin/python /home/lwecke/Github/BlackboxOptimisation/RS_BO/fix2.py

Shape of K: (1, 1) First few elements: [[1.]]
Shape of K_star: (10000, 1) First few elements: [[2.12242115e-23]
 [3.08261826e-23]
 [4.43176582e-23]]
/home/lwecke/Github/BlackboxOptimisation/RS_BO/fix2.py:103: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  offsets[i] += perturbation(x, mu, width, a, y_offset)
Shape of offsetkernel: (10000,) First few elements: [0. 0. 0.]
Shape of mu_star: (10000,) First few elements: [7.85273030e-24 1.14053565e-23 1.63970576e-23]
Shape of var_star: (10000,) First few elements: [1. 1. 1.]
[3.74540119 9.50714306]

Process finished with exit code 0

Current problematic code:

In [ ]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import norm
import matplotlib.pyplot as plt

In [ ]:
def expected_improvement(X, mu, sigma, xi=0.01):
    mu = mu.reshape(-1, 1)
    sigma = sigma.reshape(-1, 1)
    mu_sample_opt = np.max(mu)
    with np.errstate(divide='warn'):
        imp = mu - mu_sample_opt - xi
        Z = imp / sigma
        ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
        ei[sigma == 0.0] = 0.0
    return ei

def kernel(a, b, l=1.0):
    sqdist = np.sum((a[:, None, :] - b[None, :, :]) ** 2, axis=-1)
    return np.exp(-.5 * (1 / l) * sqdist)

def f(x):
    return (np.sin(x[:, 0]) + np.sin(x[:, 1])) / 5+0.5

def derivative_scalar(x):
    return (np.cos(x[:, 0]) + np.cos(x[:, 1])) / 5

def perturbation(x, mu, width, a, y_offset):
    perturbation = a * np.sum(((x - mu) / width) ** 2, axis=-1) * np.exp(-np.sum(((x - mu) / width) ** 2, axis=-1)) + y_offset
    return perturbation

def offset_function(x_train, x_test, derivative_scalar_func, PERT_WIDTH, PERT_SCALE):
    offsets = np.zeros((len(x_test), len(x_train)))
    y_offset = 0
    for i, x in enumerate(x_test):
        for j, train_point in enumerate(x_train):
            mu = train_point
            width = PERT_WIDTH
            a = PERT_SCALE * derivative_scalar_func(np.array([mu]))
            offsets[i][j] = perturbation(x, mu, width, a, y_offset)
    return offsets

def offset_kernel(X_train, X_test, derivative_scalar_func, PERT_WIDTH, PERT_SCALE):
    offset_matrix = np.zeros((len(X_test), len(X_train)))
    for i in range(len(X_test)):
        offset_matrix[i, :] = offset_function(X_train, [X_test[i]], derivative_scalar_func, PERT_WIDTH, PERT_SCALE)
    return offset_matrix

def predict(X_train, Y_train, X_test, kernel, PERT_WIDTH=3.0, PERT_SCALE=1.0):
    K = kernel(X_train, X_train)
    K_star = kernel(X_test, X_train)
    offsetkernel = offset_kernel(X_train, X_test, derivative_scalar, PERT_WIDTH, PERT_SCALE)
    mu_star = K_star @ np.linalg.inv(K) @ Y_train.flatten() + offsetkernel.flatten()
    var_star = np.diag(kernel(X_test, X_test) - K_star @ np.linalg.inv(K) @ K_star.T)
    return mu_star, var_star

if __name__ == '__main__':
    PERT_WIDTH=0.2
    PERT_SCALE=0.005
    np.random.seed(42)
    INTERVAL=10

    X_train = np.random.uniform(0, INTERVAL, size=(1, 2))
    Y_train = f(X_train)

    x1_test = np.linspace(0, INTERVAL, 100)
    x2_test = np.linspace(0, INTERVAL, 100)
    X_test = np.dstack(np.meshgrid(x1_test, x2_test)).reshape(-1, 2)

    mu_star, var_star = predict(X_train, Y_train, X_test, kernel, PERT_WIDTH=PERT_WIDTH, PERT_SCALE=PERT_SCALE)



The model looks very off. what is the issue?

Here is a working example without perturbation and offset_matrix as a reference.

In [ ]:

def expected_improvement(X, mu, sigma, xi=0.01):
    mu = mu.reshape(-1, 1)
    sigma = sigma.reshape(-1, 1)
    mu_sample_opt = np.max(mu)
    with np.errstate(divide='warn'):
        imp = mu - mu_sample_opt - xi
        Z = imp / sigma
        ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
        ei[sigma == 0.0] = 0.0
    return ei

def kernel(a, b, l=1.0):
    sqdist = np.sum((a[:, None, :] - b[None, :, :]) ** 2, axis=-1)
    return np.exp(-.5 * (1/l**2) * sqdist)

def f(x, y):
    return (np.sin(x) + np.sin(y)) / 2

def predict(x_train, y_train, x_test, kernel):
    K = kernel(x_train, x_train)
    K_star = kernel(x_test, x_train)
    mu_star = K_star @ np.linalg.inv(K) @ y_train
    var_star = np.diag(kernel(x_test, x_test) - K_star @ np.linalg.inv(K) @ K_star.T)
    return mu_star.ravel(), var_star

if __name__ == '__main__':
    n_iterations = 0
    np.random.seed(42)
    INTERVAL=10
    x_train = np.random.uniform(0, INTERVAL, size=(2, 2))
    y_train = f(x_train[:, 0], x_train[:, 1])

    x1_test = np.linspace(0, INTERVAL, 100)
    x2_test = np.linspace(0, INTERVAL, 100)
    x_test = np.dstack(np.meshgrid(x1_test, x2_test)).reshape(-1, 2)
    mu_star, var_star = predict(x_train, y_train.reshape(-1, 1), x_test, kernel)


Here is a working example for the perturbation function in 1D as a reference:

In [ ]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt

def expected_improvement(X, mu, sigma, xi=0.01):
    mu = mu.reshape(-1, 1)
    sigma = sigma.reshape(-1, 1)
    mu_sample_opt = np.max(mu)
    with np.errstate(divide='warn'):
        imp = mu - mu_sample_opt - xi
        Z = imp / sigma
        ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
        ei[sigma == 0.0] = 0.0
    return ei

def kernel(a, b, l=1.0):
    return np.exp(-0.5 * ((a - b)**2 / l**2))

def f(x):
    return (np.sin(x) + 1) / 2

def derivative_scalar(x):
    return np.cos(x)/2

def perturbation(x, mu, width, a, y_offset):
    perturbation = a * ((x - mu) / width) * np.exp(-((x - mu) / width) ** 2) + y_offset
    return perturbation

def offset_function(x_train, x_test, derivative_scalar_func, offset_range, offset_scale):
    offsets = np.zeros_like(x_test)

    for i, x in enumerate(x_test):
        for train_point in x_train:
            mu = train_point
            width = offset_range
            a = offset_scale * derivative_scalar_func(mu)
            y_offset = 0
            offsets[i] += perturbation(x, mu, width, a, y_offset)

    return offsets

def offset_kernel(x_train, x_test, derivative_scalar_func, offset_range, offset_scale):
    derivative_scalar_all = derivative_scalar_func(x_train)
    offset_matrix = np.zeros_like(x_test)

    for i in range(len(x_test)):
        offset_matrix[i] = offset_function(x_train, x_test[i], derivative_scalar_func, offset_range, offset_scale)
    return offset_matrix

def predict_and_plot(x_train, y_train, x_test, kernel, f, offset_range=3.0, offset_scale=1.0):
    K = np.zeros((len(x_train), len(x_train)))
    for i in range(len(x_train)):
        for j in range(len(x_train)):
            K[i, j] = kernel(x_train[i], x_train[j])
    K_star = np.zeros((len(x_test), len(x_train)))
    for i in range(len(x_test)):
        for j in range(len(x_train)):
            K_star[i, j] = kernel(x_test[i], x_train[j])
    offsetkernel = offset_kernel(x_train, x_test, derivative_scalar, offset_range=offset_range, offset_scale=offset_scale)
    mu_star = K_star @ np.linalg.inv(K) @ y_train.flatten() + offsetkernel.flatten()
    var_star = np.zeros(len(x_test))
    for i in range(len(x_test)):
        var_star[i] = kernel(x_test[i], x_test[i]) - K_star[i] @ np.linalg.inv(K) @ K_star[i].T
    return mu_star, var_star

if __name__ == '__main__':
    OFFSET_RANGE=1
    OFFSET_SCALE=1
    np.random.seed(42)
    INTERVAL=10
    x_train = np.random.uniform(0, INTERVAL, 2).reshape(-1, 1)

    y_train = x_train

    x_test = np.linspace(0, INTERVAL, 100).reshape(-1, 1)

    mu_star, var_star = predict_and_plot(x_train, y_train, x_test, kernel, f,offset_range=OFFSET_RANGE,offset_scale=OFFSET_SCALE)